In [1]:
!pip uninstall -y tensorflow tensorflow-cpu tensorflow-gpu tensorflow-intel keras keras-nightly keras-preprocessing keras-vis tf-nightly tf-estimator-nightly tensorflow-estimator
!pip install -q transformers datasets sacrebleu sentencepiece accelerate evaluate protobuf==3.20.3

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664

In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
import numpy as np

In [3]:
base_path = "/kaggle/input/vlsp-dataset/data"

def read_parallel(src_file, tgt_file):
    with open(base_path + src_file, encoding="utf-8") as f_src, open(base_path + tgt_file, encoding="utf-8") as f_tgt:
        src = f_src.read().strip().splitlines()
        tgt = f_tgt.read().strip().splitlines()
    n = min(len(src), len(tgt))
    return list(zip(src[:n], tgt[:n]))

vlsp_train = read_parallel("/train.en.txt", "/train.vi.txt")
print(len(vlsp_train))
vlsp_test  = read_parallel("/public_test.en.txt", "/public_test.vi.txt")
print(len(vlsp_test))
vlsp_full = set(vlsp_train + vlsp_test)

print(f"Đã load dữ liệu VLSP: {len(vlsp_full)} cặp câu.")

500000
3000
Đã load dữ liệu VLSP: 349391 cặp câu.


In [4]:
print("\nĐang tải dataset tmnam20/ViPubMed...")
full_dataset = load_dataset("tmnam20/ViPubMed", split="pubmed22")


Đang tải dataset tmnam20/ViPubMed...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/89 [00:00<?, ?it/s]

data/pubmed22-00000-of-00089-5cd8ea7c354(…):   0%|          | 0.00/254M [00:00<?, ?B/s]

data/pubmed22-00001-of-00089-a96075be264(…):   0%|          | 0.00/253M [00:00<?, ?B/s]

...

data/pubmed22-00086-of-00089-57560355290(…):   0%|          | 0.00/257M [00:00<?, ?B/s]

data/pubmed22-00087-of-00089-dd626bdefa9(…):   0%|          | 0.00/258M [00:00<?, ?B/s]

data/pubmed22-00088-of-00089-3612e1db04a(…):   0%|          | 0.00/261M [00:00<?, ?B/s]

Generating pubmed22 split:   0%|          | 0/20087006 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/87 [00:00<?, ?it/s]

In [5]:
print(f"Tổng số lượng dataset gốc: {len(full_dataset)}")
print(f"Tên các cột: {full_dataset.column_names}")

Tổng số lượng dataset gốc: 20087006
Tên các cột: ['en', 'vi']


In [6]:
print("-" * 40)
print("5 CẶP CÂU MẪU (GỐC):")
for i in range(5):
    item = full_dataset[i]
    cols = full_dataset.column_names
    en_col = 'en' 
    vi_col = 'vi'
    
    print(f"Mẫu {i+1}:")
    print(f"  🇬🇧 EN: {item[en_col][:200]}...")
    print(f"  🇻🇳 VI: {item[vi_col][:200]}...")
print("-" * 40)

----------------------------------------
5 CẶP CÂU MẪU (GỐC):
Mẫu 1:
  🇬🇧 EN: In the context of a training study, the functional relationship between discrimination and production was investigated. Four subjects were selected for study. For two subjects, Condition I consisted o...
  🇻🇳 VI: Trong nghiên cứu đào tạo, mối quan hệ chức năng giữa sự phân biệt đối xử và sản xuất được khảo sát. Bốn đối tượng được chọn tham gia nghiên cứu. Đối với hai đối tượng, Điều kiện I bao gồm đào tạo sản ...
Mẫu 2:
  🇬🇧 EN: The attachment of the bacterial chromosome to the cytoplasmic membrane in Escherichia coli was studied. The initiator DNA was specifically labeled and the outer and cytoplasmic membranes were separate...
  🇻🇳 VI: Sự gắn kết của nhiễm sắc thể vi khuẩn với màng tế bào chất ở Escherichia coli đã được nghiên cứu. DNA khởi đầu được đánh dấu cụ thể và màng ngoài và màng tế bào chất được tách ra theo một gradient suc...
Mẫu 3:
  🇬🇧 EN: The interaction between ethidium bromide and ribosomal R

In [7]:
print("\nĐang xáo trộn và lấy ngẫu nhiên 700,000 cặp...")
subset_dataset = full_dataset.shuffle(seed=42).select(range(700000))
print(f"Đã lấy xong subset: {len(subset_dataset)} dòng.")


Đang xáo trộn và lấy ngẫu nhiên 700,000 cặp...
Đã lấy xong subset: 700000 dòng.


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
from accelerate import notebook_launcher

In [9]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.3 MB/s eta 0:00:00


In [10]:
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 128

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

In [11]:
split_data = subset_dataset.train_test_split(test_size=0.01, seed=42)

def preprocess_function(examples):
    inputs = [str(x) for x in examples[en_col]]
    targets = [str(x) for x in examples[vi_col]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

print("\nĐang xử lý dữ liệu (Tokenizing)...")
tokenized_datasets = split_data.map(preprocess_function, batched=True, remove_columns=subset_dataset.column_names)


Đang xử lý dữ liệu (Tokenizing)...


Map:   0%|          | 0/693000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [12]:
args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/vipubmed_checkpoint",
    eval_strategy="epoch",        
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    save_total_limit=3,
    logging_steps=100,
    ddp_find_unused_parameters=False,
    report_to="none"
)

In [13]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [[l.strip()] for l in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [14]:
def train_fn():
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    print("Bắt đầu training...")
    trainer.train()
    
    trainer.save_model("/kaggle/working/vipubmed_checkpoint/final_model")
    tokenizer.save_pretrained("/kaggle/working/vipubmed_checkpoint/final_model")
    print("Đã lưu model xong.")

In [15]:
train_fn()

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

/tmp/ipykernel_20/1597605557.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Bắt đầu training...


model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,1.032500,0.896556,53.936929
2,0.895200,0.793191,55.880047
3,0.863100,0.768252,56.570923


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead

Đã lưu model xong.
